In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

df = pd.read_spss('r31iall_61_2.sav')

/Users/egorserockin/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Обработка данных

In [2]:
data = df[['aaj10', 'popul', 'aaj161.3y', 'aam71', 'aa_occup08',
           'aa_educ', 'aa_marst', 'aaj1.1.1',
           'aah5', 'aa_age', 'aaj319', 'aaj72.172', 'aaj21.3',
           'aal90']]
data.rename(columns={'aaj10': 'sal', 'popul': 'popul', 'aaj161.3y': 'w_years', 'aam71': 'smoke', 
                     'aa_occup08': 'occup', 'aa_educ': 'educ', 'aa_marst': 'marriage', 
                     'aaj1.1.1': 'satisf', 'aah5': 'sex', 'aa_age': 'age', 
                     'aaj319': 'school', 'aaj72.172': 'kids', 'aaj21.3': 'harm', 'aal90': 'ill'}, inplace=True)

data.reset_index(inplace=True)
data.drop('index', axis=1, inplace=True)

In [3]:
data.to_csv('data.csv')
data = pd.read_csv('data.csv')

data['sal'] = data['sal'].fillna(0)
data['w_years'] = data['w_years'].fillna(0)
data['smoke'] = data['smoke'].fillna(0)
data['occup'] = data['occup'].fillna('без квалификации')
data['educ'] = data['educ'].fillna('без образования')
data['marriage'] = data['marriage'].fillna('Никогда в браке не состояли')
data['satisf'] = data['satisf'].fillna('И ДА, И НЕТ')
data['school'] = data['school'].fillna('без образования')
data['kids'] = data['kids'].fillna(0)
data['harm'] = data['harm'].fillna('Нет')
data['ill'] = data['ill'].fillna(0)

values = ['ЗАТРУДНЯЮСЬ ОТВЕТИТЬ', 'НЕТ ОТВЕТА', 'ОТКАЗ ОТ ОТВЕТА', 'ДРУГОЕ']
data = data[data.isin(values) == False]

data.drop('Unnamed: 0', axis=1, inplace=True)

data['age'] = pd.to_numeric(data['age'])

data['w_years'][data['w_years'] == 'НЕ БЫЛО СТАЖА'] = 0 
data['w_years'] = pd.to_numeric(data['w_years'])

data['smoke'] = data['smoke'].replace('Да', 1)
data['smoke'] = data['smoke'].replace('Нет', 0)
data['smoke'] = pd.to_numeric(data['smoke'])

data['harm'] = data['harm'].replace('Да', 1)
data['harm'] = data['harm'].replace('Нет', 0)
data['harm'] = pd.to_numeric(data['harm'])

data['sex'] = data['sex'].replace('МУЖСКОЙ', 1)
data['sex'] = data['sex'].replace('ЖЕНСКИЙ', 0)
data['sex'] = pd.to_numeric(data['sex'])

data['sal'] = pd.to_numeric(data['sal'])

In [4]:
dict_marriage = {'Разведены и в браке не состоите': 'не_состоит',
                 'Состоите в зарегистрированном браке': 'состоит',
                 'Bдовец (вдова)': 'не_состоит',
                 'Никогда в браке не состояли': 'не_состоит',
                 'Живете вместе, но не зарегистрированы': 'сожительство',
                 'ОФИЦИАЛЬНО ЗАРЕГИСТРИРОВАНЫ, НО ВМЕСТЕ НЕ ПРОЖИВАЮТ': 'не_состоит'}

data.marriage = data.marriage.map(dict_marriage)


dict_educ = {'есть диплом о высшем образовании': 'ВЫСШЕЕ',
             '9 классов школы': 'ШКОЛА', 
             'среднее образование - есть аттестат о ср. образовании': 'ШКОЛА',
             '10 и более классов школы и техникум без диплома':'ПТУ',
             '7-9 классов школы (незак. среднее) и менее 2 лет в техникуме': 'ПТУ',
             'техникум с дипломом': 'ПТУ', 
             '8 классов школы': 'ШКОЛА', 
             '10 и более классов школы и какое-либо професс. обр. с дипломом': 'ПТУ',
             'аспирантура и т.п. с дипломом': 'ВЫСШЕЕ',
             '1-2 года в высшем учебном заведении': 'ШКОЛА',
             '10 и более классов школы и какое-либо професс. обр. без диплома': 'ПТУ',
             '7-9 классов школы (незак. средн) + ПТУ с дипломом': 'ПТУ',
             '3 и более лет в высшем учебном заведении':'ШКОЛА',
             '7-9 классов школы (незак. средн) + ПТУ без диплома': 'ПТУ',
             'аспирантура и т.п. без диплома': 'ВЫСШЕЕ',
             '7 классов школы': 'ШКОЛА',
             '0 классов школы': 'ШКОЛА',
             '5 классов школы': 'ШКОЛА',
             '10 и более классов школы без аттестата о среднем образовании': 'ШКОЛА',
             'без образования': 'без образования'}
data.educ = data.educ.map(dict_educ)

dict_school = {'В ОБЫЧНОЙ СРЕДНЕЙ ШКОЛЕ': 'ОБЫЧНАЯ_ШКОЛА',
               'В ВЕЧЕРНЕЙ ШКОЛЕ': 'ИНЫЕ_ВИДЫ',
               'В ШКОЛЕ ДРУГОГО ТИПА': 'ИНЫЕ_ВИДЫ',
               'В ОБЩЕОБРАЗОВАТЕЛЬНОМ ЛИЦЕЕ': 'УГЛУБ',
               'В ГИМНАЗИИ, ШКОЛЕ С ГИМНАЗИЧЕСКИМИ КЛАССАМИ': 'УГЛУБ',
               'В ШКОЛЕ С УГЛУБЛЕННЫМ ИЛИ ПРОФИЛЬНЫМ ПРЕПОДАВАНИЕМ ПРЕДМЕТОВ': 'УГЛУБ',
               'В ШКОЛЕ-ЭКСТЕРНАТЕ': 'ИНЫЕ_ВИДЫ',
               'без образования': 'без образования'}
data.school = data.school.map(dict_school)

dict_satisf = {'СКОРЕЕ УДОВЛЕТВОРЕНЫ': 'ДА',
               'ПОЛНОСТЬЮ УДОВЛЕТВОРЕНЫ': 'ДА',
               'СКОРЕЕ НЕ УДОВЛЕТВОРЕНЫ': 'НЕТ',
               'СОВСЕМ НЕ УДОВЛЕТВОРЕНЫ': 'НЕТ',
               'И ДА, И НЕТ': 'И_ДА_И_НЕТ'}
data.satisf = data.satisf.map(dict_satisf)

dict_occup = {'специалисты высшего уровня квалификации': 'ВЫСШАЯ_КВАЛИФИКАЦИЯ',
              'специалисты среднего уровня квалификации; чиновники': 'ГОС_СЛУЖАЩИЙ',
              'работники сферы торговли и услуг': 'ТОРГОВЛЯ',
              'служащие офисные и по обслуживанию клиентов': 'ОФИСНЫЕ_РАБОЧИЕ',
              'неквалифицированные рабочие всех отраслей': 'НЕКВАЛ_РАБОЧИЕ',
              'законодатели; крупные чиновники; руководители высш. и сред. звена': 'ГОС_СЛУЖАЩИЙ',
              'квалифицированные рабочие, использующие машины и механизмы': 'КВАЛ_РАБОЧИЕ',
              'квалифицированные рабочие, занятые ручным трудом': 'КВАЛ_РАБОЧИЕ',
              'квалифицированные работники сельского, лесного хоз-ва и рыбоводства': 'КВАЛ_РАБОЧИЕ',
              'военнослужащие': 'ГОС_СЛУЖАЩИЙ',
              'без квалификации': 'без_квалификации'}

data.occup = data.occup.map(dict_occup)

data.dropna(inplace=True)

data['kids'] = pd.to_numeric(data['kids'])
data['ill'] = pd.to_numeric(data['ill'])

data['popul_sq'] = data['popul'] ** 2
data['w_years_sq'] = data['w_years'] ** 2 
data['popul_ln'] = np.log(data['popul'])

cat_list = ['educ', 'marriage', 'school', 'satisf', 'occup']
data = pd.get_dummies(data, columns=cat_list) * 1
data=data.loc[:357]

In [5]:
data.to_csv('data.csv')

In [6]:
data

,sal,popul,w_years,smoke,sex,age,kids,harm,ill,popul_sq,...,satisf_ДА,satisf_И_ДА_И_НЕТ,satisf_НЕТ,occup_ВЫСШАЯ_КВАЛИФИКАЦИЯ,occup_ГОС_СЛУЖАЩИЙ,occup_КВАЛ_РАБОЧИЕ,occup_НЕКВАЛ_РАБОЧИЕ,occup_ОФИСНЫЕ_РАБОЧИЕ,occup_ТОРГОВЛЯ,occup_без_квалификации
0,47000.0,11900.0,30.0,1.0,0,49.0,2.0,1.0,0.0,1.416100e+08,...,0,1,0,1,0,0,0,0,0,0
1,30000.0,11900.0,12.0,1.0,1,36.0,2.0,0.0,0.0,1.416100e+08,...,1,0,0,0,1,0,0,0,0,0
2,0.0,11900.0,40.0,0.0,0,78.0,0.0,0.0,0.0,1.416100e+08,...,0,1,0,0,0,0,0,0,0,1
3,0.0,11900.0,46.0,0.0,0,76.0,3.0,0.0,0.0,1.416100e+08,...,0,1,0,0,0,0,0,0,0,1
4,0.0,11900.0,35.0,0.0,1,54.0,7.0,0.0,0.0,1.416100e+08,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,30000.0,1308700.0,4.0,0.0,1,27.0,0.0,0.0,0.0,1.712696e+12,...,1,0,0,1,0,0,0,0,0,0
353,28000.0,1308700.0,20.0,0.0,0,50.0,1.0,0.0,0.0,1.712696e+12,...,1,0,0,1,0,0,0,0,0,0
354,35000.0,1308700.0,26.0,0.0,1,56.0,1.0,0.0,0.0,1.712696e+12,...,1,0,0,1,0,0,0,0,0,0
356,40000.0,1308700.0,8.0,0.0,1,30.0,0.0,0.0,0.0,1.712696e+12,...,1,0,0,1,0,0,0,0,0,0


`sal` - зарплата (таргет)\
`popul` - численность населения\
`w_years` - опыт работы\
`smoke` - курение (1 - да, 0 - нет)\
`occup` - квал\
`educ` - образование\
`marriage` - семейное полоежение\
`satisf` - удовлетворенность жизнью\
`sex` - пол (1 - мужчина, 0 - женщина)\
`age` - возраст\
`school` - вид школы\
`kids` - количество детей\
`harm` - вредность производства (1 - вредно, 0 - нет)\
`ill` - количество дней, пропущенных по болезни

# Построение моделей

In [7]:
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from statsmodels.sandbox.regression.gmm import IV2SLS
from statsmodels.stats.outliers_influence import variance_inflation_factor

## Маленькая заметка всех больших проблем

Иван Павлович, здравствуйте. С этой проблемой мы поборолись на протяжении 4-х дней, перебирая все возможные спецификации модели, однако ничего не смогло помочь снизить хоть как-то мультиколлинеарность модели и получить адекватные модели, которые хотя бы строятся

In [8]:
X = add_constant(data[['popul', 'popul_ln', 'popul_sq', 'w_years', 'w_years_sq', 'smoke', 
                       'occup_ВЫСШАЯ_КВАЛИФИКАЦИЯ', 'occup_КВАЛ_РАБОЧИЕ', 'occup_НЕКВАЛ_РАБОЧИЕ',
                       'educ_ВЫСШЕЕ', 'marriage_состоит', 'age', 'ill']])


VIFs = pd.DataFrame()
VIFs['Variable'] = X.columns
VIFs['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
VIFs.to_excel('vifs.xlsx')
VIFs

,Variable,VIF
0,const,70.651449
1,popul,135.927965
2,popul_ln,3.426150
3,popul_sq,117.208519
4,w_years,22.819612
5,w_years_sq,19.326977
6,smoke,1.260087
7,occup_ВЫСШАЯ_КВАЛИФИКАЦИЯ,1.504499
8,occup_КВАЛ_РАБОЧИЕ,1.303396
9,occup_НЕКВАЛ_РАБОЧИЕ,1.086832


## OLS

In [9]:
model = OLS(data['sal'], add_constant(data[['popul', 'popul_ln', 'popul_sq', 'w_years', 'w_years_sq', 
                                            'smoke', 'occup_ВЫСШАЯ_КВАЛИФИКАЦИЯ', 'occup_КВАЛ_РАБОЧИЕ',
                                            'occup_НЕКВАЛ_РАБОЧИЕ', 'educ_ВЫСШЕЕ', 'marriage_состоит', 'age',
                                            'ill']])).fit()

In [18]:
model.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                          Results: Ordinary least squares
===================================================================================
Model:                    OLS                   Adj. R-squared:          0.469     
Dependent Variable:       sal                   AIC:                     7058.2728 
Date:                     2024-05-11 15:46      BIC:                     7110.8532 
No. Observations:         316                   Log-Likelihood:          -3515.1   
Df Model:                 13                    F-statistic:             22.42     
Df Residuals:             302                   Prob (F-statistic):      3.87e-37  
R-squared:                0.491                 Scale:                   2.8136e+08
-----------------------------------------------------------------------------------
                            Coef.     Std.Err.    t    P>|t|    [0.025     0.975]  
-----------------------------------------------------------------------------------
const                     33328.6226 7931.3346  4.2021 0.0000 17720.9439 48936.3012
popul                         0.0187    0.0227  0.8253 0.4098    -0.0259     0.0633
popul_ln                   -333.3458  598.7702 -0.5567 0.5781 -1511.6358   844.9443
popul_sq                     -0.0000    0.0000 -0.4534 0.6506    -0.0000     0.0000
w_years                    1374.3306  357.3951  3.8454 0.0001   671.0307  2077.6306
w_years_sq                  -17.7358    6.0813 -2.9165 0.0038   -29.7028    -5.7688
smoke                      6093.3545 2667.2291  2.2845 0.0230   844.6471 11342.0620
occup_ВЫСШАЯ_КВАЛИФИКАЦИЯ 14351.0936 3836.4943  3.7407 0.0002  6801.4475 21900.7397
occup_КВАЛ_РАБОЧИЕ        21245.8373 3173.2999  6.6952 0.0000 15001.2583 27490.4163
occup_НЕКВАЛ_РАБОЧИЕ       5419.1982 5366.7066  1.0098 0.3134 -5141.6765 15980.0729
educ_ВЫСШЕЕ                5402.7542 2737.5161  1.9736 0.0493    15.7326 10789.7757
marriage_состоит           -107.0565 1987.6616 -0.0539 0.9571 -4018.4768  3804.3638
age                        -798.6536  120.0291 -6.6538 0.0000 -1034.8529  -562.4543
ill                          50.3393  128.9734  0.3903 0.6966  -203.4611   304.1397
-----------------------------------------------------------------------------------
Omnibus:                  171.892          Durbin-Watson:             1.738        
Prob(Omnibus):            0.000            Jarque-Bera (JB):          1232.508     
Skew:                     2.168            Prob(JB):                  0.000        
Kurtosis:                 11.649           Condition No.:             5846562826171
===================================================================================
* The condition number is large (6e+12). This might indicate             strong
multicollinearity or other numerical problems.
"""

In [11]:
with open('results_ols.tex', 'w') as file:
    file.write(model.summary2().as_latex())

## 2SLS

In [12]:
target = data['sal']
X = add_constant(data[['popul', 'popul_ln', 'popul_sq', 'w_years', 'w_years_sq', 'smoke', 'occup_ВЫСШАЯ_КВАЛИФИКАЦИЯ', 
                       'occup_КВАЛ_РАБОЧИЕ', 'occup_НЕКВАЛ_РАБОЧИЕ', 'educ_ВЫСШЕЕ', 'marriage_состоит', 'age',
                       'ill']])
IV = data[['satisf_И_ДА_И_НЕТ', 'satisf_НЕТ', 'sex', 'school_УГЛУБ', 'kids', 'harm']]
model_iv = IV2SLS(target, X, X[['popul', 'const', 'popul_ln', 'popul_sq', 'w_years','w_years_sq', 'age', 'ill']
                              ].join(IV)).fit()

In [13]:
model_iv.summary2()

/Users/egorserockin/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 13, but rank is 12
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary2.Summary'>
"""
                                   Results: IV2SLS
======================================================================================
Model:                     IV2SLS                 R-squared:                0.049     
Dependent Variable:        sal                    Adj. R-squared:           0.008     
Date:                      2024-05-11 15:42       F-statistic:              13.92     
No. Observations:          316                    Prob (F-statistic):       2.93e-24  
Df Model:                  13                     Scale:                    5.2606e+08
Df Residuals:              302                                                        
--------------------------------------------------------------------------------------
                            Coef.     Std.Err.     t    P>|t|     [0.025      0.975]  
--------------------------------------------------------------------------------------
const                     -1340.4239 26992.3525 -0.0497 0.9604 -54457.3309  51776.4831
popul                         0.0162     0.0919  0.1764 0.8601     -0.1646      0.1971
popul_ln                  -1124.9146  1800.9766 -0.6246 0.5327  -4668.9668   2419.1376
popul_sq                     -0.0000     0.0000 -0.1392 0.8894     -0.0000      0.0000
w_years                     574.6506  1010.2583  0.5688 0.5699  -1413.3864   2562.6876
w_years_sq                  -10.4041    19.2520 -0.5404 0.5893    -48.2891     27.4809
smoke                     10400.9679 29336.7343  0.3545 0.7232 -47329.3313  68131.2672
occup_ВЫСШАЯ_КВАЛИФИКАЦИЯ 35801.2873 66746.1534  0.5364 0.5921 -95545.1460 167147.7205
occup_КВАЛ_РАБОЧИЕ        44307.0970 37004.9087  1.1973 0.2321 -28513.0209 117127.2149
occup_НЕКВАЛ_РАБОЧИЕ      62458.4524 58510.2061  1.0675 0.2866 -52680.8700 177597.7749
educ_ВЫСШЕЕ               26102.8119 51077.0470  0.5110 0.6097 -74409.1667 126614.7906
marriage_состоит           8666.6286 14507.7255  0.5974 0.5507 -19882.4023  37215.6595
age                         -80.7196   301.0309 -0.2681 0.7888   -673.1034    511.6641
ill                         -90.1316   288.7235 -0.3122 0.7551   -658.2962    478.0330
--------------------------------------------------------------------------------------
Omnibus:                  32.598            Durbin-Watson:               1.761        
Prob(Omnibus):            0.000             Jarque-Bera (JB):            73.878       
Skew:                     0.518             Prob(JB):                    0.000        
Kurtosis:                 5.131             Condition No.:               5844912789236
======================================================================================
* The condition number is large (6e+12). This might indicate             strong
multicollinearity or other numerical problems.
"""

In [14]:
with open('results_2sls.tex', 'w') as file:
    file.write(model_iv.summary2().as_latex())

### Проведем тест Ву-Хаусмана

К сожалению, использование IV2SLS только из одной библиотеки не представляется возможным, так как и __statsmodels__ имеет преимущество (выведение результирующей таблицы без экспоненциальноый записи), так и __linearmodels__ (проведение теста Хаусмана и выведение красивого результата)

In [15]:
from linearmodels.iv.model import IV2SLS

target = data['sal']
X = add_constant(data[['popul', 'popul_ln', 'popul_sq', 'w_years', 'w_years_sq', 'smoke', 
                       'occup_ВЫСШАЯ_КВАЛИФИКАЦИЯ', 'occup_КВАЛ_РАБОЧИЕ', 'occup_НЕКВАЛ_РАБОЧИЕ',
                       'educ_ВЫСШЕЕ', 'marriage_состоит', 'age', 'ill']])
IV = data[['satisf_И_ДА_И_НЕТ', 'satisf_НЕТ', 'sex', 'school_УГЛУБ', 'kids', 'harm']]
model_iv_WH = IV2SLS(dependent=target, 
                  exog=X[['const','popul', 'popul_ln', 'popul_sq', 'w_years','w_years_sq', 'age', 'ill']],
                  endog=X.drop(['const','popul', 'popul_ln', 'popul_sq', 'w_years','w_years_sq', 'age', 'ill'], axis=1),
                  instruments=IV).fit()

model_iv_WH.wu_hausman()

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 11.8058
P-value: 0.0000
Distributed: F(6,296)
WaldTestStatistic, id: 0x12fa42b50

## Общая таблица

In [16]:
from statsmodels.iolib.summary2 import summary_col


with open('results_overall.tex', 'w') as file:
    file.write(summary_col([model, model_iv], model_names=('OLS', '2SLS')).as_latex())
    
summary_col([model, model_iv], model_names=('OLS', '2SLS'))

,OLS,2SLS
R-squared,0.4912,0.0486
R-squared Adj.,0.4693,0.0077
age,-798.6536,-80.7196
,(120.0291),(301.0309)
const,33328.6226,-1340.4239
,(7931.3346),(26992.3525)
educ_ВЫСШЕЕ,5402.7542,26102.8119
,(2737.5161),(51077.0470)
ill,50.3393,-90.1316
,(128.9734),(288.7235)
